In [2]:
import os
import git
from pathlib import Path

ROOT_DIR =  Path(git.Repo('.', search_parent_directories=True).working_tree_dir)
SAVE_FIGS = False

In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
from reporting import *
plots_path = os.path.join(ROOT_DIR, "publication", "paper", "draft_plots")
save_path_with_hull = Path(os.path.join(ROOT_DIR, 'publication', 'paper', 'CSVs', 'final_results_with_hull.pickle'))
main_df = pd.read_pickle(save_path_with_hull)
RERUN = False
np.random.seed(0)

In [4]:
name_mapping = {'syntheticMRI3Dfull': 'syntheticMRI3D',
                'syntheticMRI2Daxial': 'syntheticMRI2D (axial)',
                'syntheticMRI2Dsagittal': 'syntheticMRI2D (sagittal)',
                'syntheticMRI2Dcoronal': 'syntheticMRI2D (coronal)',
                'spaceNetfull': 'spaceNet',
                'cocoindoor': 'coco (indoor)',
                'cocooutdoor': 'coco (outdoor)',
                'pastisfull': 'pastis',
                'segmentAnythingfull': 'segmentAnything',
                'agriVisionfull': 'agriVision',
                'standardTestingfull': 'standardTesting'}

main_df['dataset'] = (main_df['dataset'] + main_df['subset']).replace(name_mapping)

In [5]:
granularity = ['transform', 'dataset_type', 'dataset']
temp = main_df.copy()[(main_df['dataset'] != 'standardTesting')] #  & (main_df['transform'] != 'learned')
temp['practically_pass'] = (temp['failure_category'] == 'practically_pass')
temp['actually_pass'] = (temp['failure_category'] == 'actually_pass')
summary_df = (temp.groupby(granularity)[['actually_pass','practically_pass','intersect_roi']].mean() * 100).round(1)
summary_df

actually_pass  \
transform dataset_type   dataset                                    
fourier   remote sensing agriVision                           2.3   
                         pastis                              90.9   
                         spaceNet                             0.0   
learned   natural        coco (indoor)                        0.0   
                         coco (outdoor)                       0.0   
                         segmentAnything                      0.0   
          remote sensing agriVision                           0.0   
                         pastis                               0.0   
                         spaceNet                             0.0   
wavelet   medical        syntheticMRI2D (axial)               0.0   
                         syntheticMRI2D (coronal)             8.3   
                         syntheticMRI2D (sagittal)            0.0   
                         syntheticMRI3D                       6.1   
          natural        coco (indoor)                       25.0   
                         coco (outdoor)                      25.0   
                         segmentAnything                     25.0   
          remote sensing agriVision                          13.9   
                         pastis                              57.1   
                         spaceNet                             4.7   

                                                    practically_pass  \
transform dataset_type   dataset                                       
fourier   remote sensing agriVision                             93.2   
                         pastis                                  9.1   
                         spaceNet                              100.0   
learned   natural        coco (indoor)                          56.5   
                         coco (outdoor)                         59.4   
                         segmentAnything                        73.4   
          remote sensing agriVision                             78.3   
                         pastis                                 85.7   
                         spaceNet                               42.9   
wavelet   medical        syntheticMRI2D (axial)                 66.7   
                         syntheticMRI2D (coronal)               62.5   
                         syntheticMRI2D (sagittal)              70.8   
                         syntheticMRI3D                         53.1   
          natural        coco (indoor)                          25.0   
                         coco (outdoor)                         28.1   
                         segmentAnything                        60.2   
          remote sensing agriVision                             86.1   
                         pastis                                 42.9   
                         spaceNet                               48.4   

                                                    intersect_roi  
transform dataset_type   dataset                                   
fourier   remote sensing agriVision                          70.5  
                         pastis                             100.0  
                         spaceNet                            50.0  
learned   natural        coco (indoor)                       45.2  
                         coco (outdoor)                      26.6  
                         segmentAnything                     51.6  
          remote sensing agriVision                          51.7  
                         pastis                              77.1  
                         spaceNet                            48.2  
wavelet   medical        syntheticMRI2D (axial)              33.3  
                         syntheticMRI2D (coronal)            57.1  
                         syntheticMRI2D (sagittal)           76.2  
                         syntheticMRI3D                      66.7  
          natural        coco (indoor)                       37.5  
 

In [6]:
granularity = ['dataset', 'transform']
temp = main_df.copy()# & (main_df['transform'] != 'learned')]
temp['practically_pass'] = (temp['failure_category'] == 'practically_pass')
temp['actually_pass'] = (temp['failure_category'] == 'actually_pass')
temp['borderline'] = (temp['failure_category'] == 'borderline')
summary_df = (temp.groupby(granularity)[['actually_pass','practically_pass','intersect_roi']].mean() * 100).round(1).sort_values('transform')
summary_df.reset_index()#.drop('dataset', axis=1)

,dataset,transform,actually_pass,practically_pass,intersect_roi
0,agriVision,fourier,2.3,93.2,70.5
1,spaceNet,fourier,0.0,100.0,50.0
2,pastis,fourier,90.9,9.1,100.0
3,standardTesting,learned,0.0,24.1,70.4
4,spaceNet,learned,0.0,42.9,48.2
5,pastis,learned,0.0,85.7,77.1
6,segmentAnything,learned,0.0,73.4,51.6
7,coco (outdoor),learned,0.0,59.4,26.6
8,coco (indoor),learned,0.0,56.5,45.2
9,agriVision,learned,0.0,78.3,51.7


In [7]:
save_path = Path(os.path.join(ROOT_DIR, 'publication', 'poster', 'CSVs', 'summary_table.csv'))
summary_df.to_csv(save_path) #.drop('learned')

In [8]:
summary_df = summary_df.reset_index()
summary_df

,dataset,transform,actually_pass,practically_pass,intersect_roi
0,agriVision,fourier,2.3,93.2,70.5
1,spaceNet,fourier,0.0,100.0,50.0
2,pastis,fourier,90.9,9.1,100.0
3,standardTesting,learned,0.0,24.1,70.4
4,spaceNet,learned,0.0,42.9,48.2
5,pastis,learned,0.0,85.7,77.1
6,segmentAnything,learned,0.0,73.4,51.6
7,coco (outdoor),learned,0.0,59.4,26.6
8,coco (indoor),learned,0.0,56.5,45.2
9,agriVision,learned,0.0,78.3,51.7


In [9]:
(temp[['actually_pass','practically_pass','intersect_roi', 'borderline']].mean(numeric_only=True)*100).round(1)

actually_pass       14.5
practically_pass    54.2
intersect_roi       52.7
borderline           5.7
dtype: float64

In [10]:
(temp.groupby(['transform'])[['kstest_stat_best', 'actually_pass','practically_pass','intersect_roi', 'total_samples']].median(numeric_only=True)*1)#.round(1)

,kstest_stat_best,actually_pass,practically_pass,intersect_roi,total_samples,total_samples
transform,,,,,,
fourier,0.003361,0.0,1.0,1.0,552780.0,552780.0
learned,0.008877,0.0,1.0,1.0,196608000.0,196608000.0
wavelet,0.007165,0.0,0.0,0.0,452608.0,452608.0


In [11]:
(temp.groupby(['dataset', 'transform'])[['kstest_stat_best', 'actually_pass','practically_pass','intersect_roi', 'total_samples']].median(numeric_only=True)*1)#.round(1)

kstest_stat_best  actually_pass  \
dataset                   transform                                    
agriVision                fourier            0.004708            0.0   
                          learned            0.005417            0.0   
                          wavelet            0.005020            0.0   
coco (indoor)             learned            0.009850            0.0   
                          wavelet            0.013062            0.0   
coco (outdoor)            learned            0.008832            0.0   
                          wavelet            0.011024            0.0   
pastis                    fourier            0.001526            1.0   
                          learned            0.006469            0.0   
                          wavelet            0.004502            1.0   
segmentAnything           learned            0.004959            0.0   
                          wavelet            0.004032            0.0   
spaceNet                  fourier            0.005150            0.0   
                          learned            0.012170            0.0   
                          wavelet            0.011693            0.0   
standardTesting           learned            0.013850            0.0   
                          wavelet            0.019365            0.0   
syntheticMRI2D (axial)    wavelet            0.007651            0.0   
syntheticMRI2D (coronal)  wavelet            0.004316            0.0   
syntheticMRI2D (sagittal) wavelet            0.005115            0.0   
syntheticMRI3D            wavelet            0.004573            0.0   

                                     practically_pass  intersect_roi  \
dataset                   transform                                    
agriVision                fourier                 1.0            1.0   
                          learned                 1.0            1.0   
                          wavelet                 1.0            1.0   
coco (indoor)             learned                 1.0            0.0   
                          wavelet                 0.0            0.0   
coco (outdoor)            learned                 1.0            0.0   
                          wavelet                 0.0            0.0   
pastis                    fourier                 0.0            1.0   
                          learned                 1.0            1.0   
                          wavelet                 0.0            1.0   
segmentAnything           learned                 1.0            1.0   
                          wavelet                 1.0            0.0   
spaceNet                  fourier                 1.0            0.5   
                          learned                 0.0            0.0   
                          wavelet                 0.0            0.0   
standardTesting           learned                 0.0            1.0   
                          wavelet                 0.0            0.0   
syntheticMRI2D (axial)    wavelet                 1.0            0.0   
syntheticMRI2D (coronal)  wavelet                 1.0            1.0   
syntheticMRI2D (sagittal) wavelet                 1.0            1.0   
syntheticMRI3D            wavelet                 1.0            1.0   

                                     total_samples  total_samples  
dataset                   transform                                
agriVision                fourier        2574000.0      2574000.0  
                          learned      786432000.0    786432000.0  
                          wavelet        1728000.0      1728000.0  
coco (indoor)             learned      196608000.0    196608000.0  
                          wavelet         256640.0       256640.0  
coco (outdoor)            learned      196608000.0    196608000.0  
                          wavelet         391360.0       391360.0  
pastis                    fourier         257580.0       257580.0  
                          learned         4

In [12]:
granularity = ['dataset', 'transform']
temp = main_df.copy()# & (main_df['transform'] != 'learned')]
temp['practically_pass'] = (temp['failure_category'] == 'practically_pass')
temp['actually_pass'] = (temp['failure_category'] == 'actually_pass')
temp['borderline'] = (temp['failure_category'] == 'borderline')
temp["interesting_failure"] = (temp['failure_category'] == 'interesting_failure')
temp["trivial_failure"] = (temp['failure_category'] == 'trivial_failure')
summary_df = (temp.groupby(granularity)[['actually_pass','practically_pass', "borderline", 'interesting_failure', 'trivial_failure', 'intersect_roi']].mean() * 100).round(1).sort_values('transform')
summary_df["kstest_stat_best"] = temp.groupby(granularity)['kstest_stat_best'].median(numeric_only=True)
summary_df = summary_df[['kstest_stat_best', 'actually_pass', 'practically_pass', 'borderline', 'interesting_failure', 'trivial_failure', 'intersect_roi']]
#ummary_df.reset_index()#.drop('dataset', axis=1)

In [17]:
granularity = ['dataset_type', 'dataset', 'transform']
temp = main_df.copy()# & (main_df['transform'] != 'learned')]
temp['orientation'] = temp.apply(
    lambda row: 'fourier' if row['transform'] == 'fourier' 
    else ('learned' if row['transform'] == 'learned' else row['orientation']),
    axis=1
)
temp['practically_pass'] = (temp['failure_category'] == 'practically_pass')
temp['actually_pass'] = (temp['failure_category'] == 'actually_pass')
temp['borderline'] = (temp['failure_category'] == 'borderline')
temp["interesting_failure"] = (temp['failure_category'] == 'interesting_failure')
temp["trivial_failure"] = (temp['failure_category'] == 'trivial_failure')
summary_df = (temp.groupby(granularity)[['actually_pass','practically_pass', "borderline", 'interesting_failure', 'trivial_failure', 'intersect_roi']].mean() * 100).round(1).sort_values('dataset')
summary_df["kstest_stat_best"] = temp.groupby(granularity)['kstest_stat_best'].median(numeric_only=True)
summary_df["hull_area_mean"] = temp.groupby(granularity)['hull_area'].mean(numeric_only=True)
summary_df = summary_df[['kstest_stat_best', 'actually_pass', 'practically_pass', 'borderline', 'interesting_failure', 'trivial_failure', 'intersect_roi', 'hull_area_mean']]
summary_df#.reset_index()#.drop('dataset', axis=1)

kstest_stat_best  \
dataset_type   dataset                   transform                     
remote sensing agriVision                fourier            0.004708   
                                         learned            0.005417   
                                         wavelet            0.005020   
natural        coco (indoor)             learned            0.009850   
                                         wavelet            0.013062   
               coco (outdoor)            learned            0.008832   
                                         wavelet            0.011024   
remote sensing pastis                    wavelet            0.004502   
                                         learned            0.006469   
                                         fourier            0.001526   
natural        segmentAnything           learned            0.004959   
                                         wavelet            0.004032   
remote sensing spaceNet                  wavelet            0.011693   
                                         fourier            0.005150   
                                         learned            0.012170   
natural        standardTesting           learned            0.013850   
                                         wavelet            0.019365   
medical        syntheticMRI2D (axial)    wavelet            0.007651   
               syntheticMRI2D (coronal)  wavelet            0.004316   
               syntheticMRI2D (sagittal) wavelet            0.005115   
               syntheticMRI3D            wavelet            0.004573   

                                                    actually_pass  \
dataset_type   dataset                   transform                  
remote sensing agriVision                fourier              2.3   
                                         learned              0.0   
                                         wavelet             13.9   
natural        coco (indoor)             learned              0.0   
                                         wavelet             25.0   
               coco (outdoor)            learned              0.0   
                                         wavelet             25.0   
remote sensing pastis                    wavelet             57.1   
                                         learned              0.0   
                                         fourier             90.9   
natural        segmentAnything           learned              0.0   
                                         wavelet             25.0   
remote sensing spaceNet                  wavelet              4.7   
                                         fourier              0.0   
                                         learned              0.0   
natural        standardTesting           learned              0.0   
                                         wavelet              7.4   
medical        syntheticMRI2D (axial)    wavelet              0.0   
               syntheticMRI2D (coronal)  wavelet              8.3   
               syntheticMRI2D (sagittal) wavelet              0.0   
               syntheticMRI3D            wavelet              6.1   

                                                    practically_pass  \
dataset_type   dataset                   transform                     
remote sensing agriVision                fourier                93.2   
                                         learned                78.3   
                                         wavelet                86.1   
natural        coco (indoor)             learned                56.5   
                                         wavelet                25.0   
               coco (outdoor)            learned                59.4   
                                         wavelet                28.1   
remote sensing pastis                    wavelet                42.9   
                                         learned                85.7   
               

In [ ]:
temp = main_df.copy()
